In [1]:
import sys
sys.path.insert(0, '../utils')

from dataGenerator import *

%reload_ext autoreload
%autoreload 2

In [4]:
arrayDir = '/home/mark/DataBank/PROBA-V-CHKPT/dataset'
rawDataDir = '/home/mark/DataBank/probav_data'

loadAndSaveRawData(rawDataDir, arrayDir, 'NIR', isGrayScale=True)

[ INFO ] Loading HR masks and dumping  : 100%|██████████| 566/566 [00:00<00:00, 3296.20it/s]


In [5]:
loadAndSaveRawData(rawDataDir, arrayDir, 'RED', isGrayScale=True)

[ INFO ] Loading HR masks and dumping  : 100%|██████████| 594/594 [00:00<00:00, 2412.74it/s]
